In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sqlite3
import numpy as np
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pickle
import cv2

In [5]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.data_block import *

In [6]:
import matplotlib.pyplot as plt

In [7]:
from helper.nms_grid import non_max_suppression_by_grid
from helper.nms import non_max_suppression_fast
from helper.nms_prob import non_max_suppression
from helper.nms_center_distance import non_max_suppression_by_distance

In [8]:
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
from Evaluator import *
from utils import *

In [13]:
path = Path('/data/Datasets/EIPH_WSI/')

In [14]:
database = Database()
database.open(str(path/'EIPH.sqlite'))

True

In [15]:
files = []
lbl_bbox = []

file_paths = []

getslides = """SELECT uid, filename FROM Slides"""
for currslide, filename in tqdm(database.execute(getslides).fetchall()):
    database.loadIntoMemory(currslide)

    check = True if 'erliner' in filename else False
    slidetype = 'Berliner Blau/' if check else 'Turnbull Blue/'

    slide_path = path / slidetype / filename

    down_factor = 1

    classes = {3: 0, 4: 1, 5: 2, 6: 3, 7: 4}#{3: 1, 4: 2, 5: 3, 6: 4, 7: 5} #
    labels, bboxes = [], []
    for id, annotation in database.annotations.items():
        if annotation.labels[0].classId in classes:
            d = 2 * annotation.r / down_factor
            x_min = (annotation.x1 - annotation.r) / down_factor
            y_min = (annotation.y1 - annotation.r) / down_factor
            x_max = x_min + d
            y_max = y_min + d
            label = classes[annotation.labels[0].classId]

            bboxes.append([int(x_min), int(y_min), int(x_max), int(y_max)])
            labels.append(label)

    if len(bboxes) > 0:
        lbl_bbox.append([bboxes, labels])
        files.append(slide_path)
        
        file_paths.append(slide_path)

img2bbox = dict(zip(files, np.array(lbl_bbox)))
get_y_func = lambda o:img2bbox[o]

  0%|          | 0/24 [00:00<?, ?it/s]

Loading DB into memory ...


  4%|▍         | 1/24 [00:00<00:10,  2.22it/s]

Loading DB into memory ...


  8%|▊         | 2/24 [00:01<00:15,  1.40it/s]

...

Loading DB into memory ...
Loading DB into memory ...
Loading DB into memory ...


 88%|████████▊ | 21/24 [00:05<00:00,  5.13it/s]

Loading DB into memory ...
Loading DB into memory ...


100%|██████████| 24/24 [00:06<00:00,  3.67it/s]

Loading DB into memory ...


In [16]:
file_paths

[PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/01_EIPH_563479 Berliner Blau.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/02_EIPH_574162 berliner blau.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/03_EIPH_566933 R Berliner Blau neu.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/04_EIPH_567017 Berliner Blau.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/05_EIPH_569923 Berliner Blau.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/27_EIPH_571557 berliner blau.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/31_EIPH_568354 Berliner Blau.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Berliner Blau/11_EIPH_575697 Berliner Blau.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Turnbull Blue/29_EIPH_566481 L Turnbull blue.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Turnbull Blue/28_EIPH_569948 L Turnbull blue.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Turnbull Blue/14_EIPH_568381 Turnbull blue.svs'),
 PosixPath('/data/Datasets/EIPH_WSI/Berliner

In [17]:
boxes_gt, labels_gt = get_y_func(files[0])
boxes_gt, labels_gt = np.array(boxes_gt), np.array(labels_gt)
boxes_gt[:, 2:] = boxes_gt[:, 2:] - boxes_gt[:, :2]

In [18]:
def calc_ap_fake(boxes_gt, labels_gt, error_percentage:float = 0.2,file:str="temp.png"):
    boundingBoxes = BoundingBoxes()
    evaluator = Evaluator()
    
    for box, cla in zip(boxes_gt, labels_gt):
        temp = BoundingBox(imageName=str(file), classId=cla, x=box[0],
                                       y=box[1],
                                       w=box[2], h=box[3], typeCoordinates=CoordinatesType.Absolute,
                                       bbType=BBType.GroundTruth, format=BBFormat.XYWH, imgSize=(5000, 5000))
        boundingBoxes.addBoundingBox(temp)

    for box, cla in zip(boxes_gt, labels_gt):
        
        if (np.random.choice([0,1], 1, p=[1-error_percentage, error_percentage])[0] == 1):
            cla = 2 if cla == 3 else 3
        
        temp = BoundingBox(imageName=str(file), classId=cla, x=box[0],
                                       y=box[1], classConfidence=1,
                                       w=box[2], h=box[3], typeCoordinates=CoordinatesType.Absolute,
                                       bbType=BBType.Detected, format=BBFormat.XYWH, imgSize=(5000, 5000))
        boundingBoxes.addBoundingBox(temp)

    metricsPerClass = evaluator.GetPascalVOCMetrics(boundingBoxes, 0.5)
    return np.mean([mc['AP'] for mc in metricsPerClass])


In [37]:
calc_ap_fake(bboxe, labels_gt, error_percentage=0.15)

0.737509783268065

In [38]:
calc_ap_fake(boxes_gt, labels_gt, error_percentage=0.2)

0.6804608515344162

In [39]:
calc_ap_fake(boxes_gt, labels_gt, error_percentage=0.25)

0.621609910635306

In [13]:
#  68
calc_ap_fake(boxes_gt, labels_gt, error_percentage=0.32)

0.5751201495425893

In [14]:
# 86
calc_ap_fake(boxes_gt, labels_gt, error_percentage=0.14)

0.746425555803438

In [15]:
# 73
calc_ap_fake(boxes_gt, labels_gt, error_percentage=0.27)

0.5978155286479712

In [21]:
for i in [0.688253, 0.736446, 0.734438, 0.789659, 0.693273, 0.727912, 0.719880, 0.719588, 0.864960]:
    print(calc_ap_fake(boxes_gt, labels_gt, error_percentage=1 - i))
    

0.555648311300718
0.6094898003179081
0.5988789197770366
0.6422637321865917
0.5591384403338612
0.5877368885347043
0.5713938892068777
0.575805047765334
0.7567775629667628


In [22]:
for i in [0.742972 ,0.782521, 0.693273, 0.660141, 0.729715, 0.730422, 0.732932, 0.716000, 0.811245]:
    print(calc_ap_fake(boxes_gt, labels_gt, error_percentage=1 - i))
    


0.617023167535347
0.6466077838032882
0.5666047153669802
0.5122840233922858
0.5974539715114326
0.5954900329670574
0.6075822364890735
0.5946624576835452
0.6819706288198658
